In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder object with sparse_output argument
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder on the Brand feature
encoded_brand = encoder.fit_transform(df[['Brand']])

# Combine encoded Brand with numerical features (assuming you have some)
combined_features = pd.concat([encoded_brand, df[['Category']]], axis=1)

# Use combined_features for KMeans clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(combined_features)

In [ ]:
async def get_weather(city):
    async with python_weather.Client(unit=python_weather.IMPERIAL) as client:
        weather = await client.get(city)
        return weather

# Assuming you have a DataFrame 'historic_data' with a 'date' column
sales['weather_type'] = None  # Initialize the new column
sales['SaleDate'] = pd.to_datetime(sales['SaleDate'])
# Loop through each row in the DataFrame
for index, row in sales.iterrows():
    date = row['SaleDate']
    
    # Convert the date to a string in the format used by the weather API
    date_str = date.strftime('%Y-%m-%d')
    
    # Use the get_weather function to fetch weather data for the date
    weather = asyncio.run(get_weather('Glenwood Springs'))  # Replace 'New York' with your city

    # Extract the weather information you are interested in (e.g., description, main, etc.)
    weather_type = weather.current.weather['description']
    
    # Assign the weather type to the corresponding row in the DataFrame
    sales.at[index, 'weather_type'] = weather_type

# Display the updated DataFrame
sales.head()

In [2]:
def is_holiday(x): 
    usa_holidays = holidays.UnitedStates()  # Use 'UnitedStates' instead of 'country_holidays('USA')'
    if x in usa_holidays:  # Check if the date is in the list of holidays
        return 1
    else:
        return 0

sales['holidays'] = sales['SaleDate'].apply(is_holiday) 

NameError: name 'sales' is not defined